In [1]:
import numpy as np
import pandas as pd

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns毎に処理
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Loading Data

In [3]:
data_dir = "./data/"
calendar = pd.read_csv(data_dir+'calendar.csv')
calendar = reduce_mem_usage(calendar)
print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))

sell_prices = pd.read_csv(data_dir+'sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)
print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))

sales = pd.read_csv(data_dir+'sales_train_validation.csv')
print('Sales train validation has {} rows and {} columns'.format(sales.shape[0], sales.shape[1]))

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns


In [4]:
sell_prices["store_id"].unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3'], dtype=object)

In [5]:
idCols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
product = sales[idCols].drop_duplicates()

In [6]:
import warnings
warnings.filterwarnings('ignore')

submission = pd.read_csv(data_dir+'sample_submission.csv')
validate_submission = submission[submission.id.str.endswith('validation')]
eval_submission = submission[submission.id.str.endswith('evaluation')]

# change column name
newcolumns = ["id"] + ["d_{}".format(i) for i in range(1914, 1914+28)]
validate_submission.columns = newcolumns
validate_submission = validate_submission.merge(product, how = 'left', on = 'id')

newcolumns = ["id"] + ["d_{}".format(i) for i in range(1942, 1942+28)]
eval_submission.columns = newcolumns
eval_submission['id'] = eval_submission['id'].str.replace('_evaluation','_validation')
eval_submission = eval_submission.merge(product, how = 'left', on = 'id')

# Select Subset of Sales

In [8]:
idCols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

# Use only one year of data
DAYS = 365; LAST_DAY=1913
dayCols = ["d_{}".format(i) for i in range(LAST_DAY-DAYS+1, LAST_DAY+1)]
print(len(dayCols), dayCols[0])
sales = sales[idCols+dayCols]
print(sales.shape)
sales.head()

365 d_1549
(30490, 371)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1549,d_1550,d_1551,d_1552,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,1,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,3,0,2,1,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,2,1,2,...,2,1,1,0,1,1,2,2,2,4


# Melted & Merge

In [9]:


def melted(df, name=""):
    df = pd.melt(df, id_vars = idCols, var_name = 'day', value_name = 'demand')
    print('{}: {} rows and {} columns'.format(name, df.shape[0], df.shape[1]))
    df = reduce_mem_usage(df)
    # df.to_csv(name+".csv")
    return df

melted_sales = melted(sales)
melted_sales["part"] = "train"
melted_validate = melted(validate_submission)
melted_validate["part"] = "validate"
melted_eval = melted(eval_submission)
melted_eval["part"] = "evaluate"

: 11128850 rows and 8 columns
Mem. usage decreased to 615.57 Mb (9.4% reduction)
: 853720 rows and 8 columns
Mem. usage decreased to 46.41 Mb (10.9% reduction)
: 853720 rows and 8 columns
Mem. usage decreased to 46.41 Mb (10.9% reduction)


In [10]:
# data = melted_sales
data = pd.concat([melted_sales, melted_validate, melted_eval], axis = 0)
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0,train
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0,train
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0,train
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,3,train
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1549,0,train


In [11]:
# import gc
# del melted_sales, melted_validate, melted_eval
# del sales
# gc.collect()

In [12]:
# merge with calendar, sell_prices

calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
data.drop(['d', 'day'], inplace = True, axis = 1)

print('Our dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))

# get the sell price data (this feature should be very important)
data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'])

print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))

Our dataset to train has 12836290 rows and 17 columns
Our final dataset to train has 12804253 rows and 18 columns


# Feature Engineering

In [13]:
# Encode Events Columns
import fasttext

# Use Fasttext to embed event's name and use PCA to reduce word vectors's dimention

# model = fasttext.load_model("models/requirement_text.bin")
# eventNames = data["event_name_1"].unique()
# eventNames = eventNames[1:]
# eventNames

In [14]:
# _temp = data.drop(["event_name_1", "event_name_2", "event_type_1", "event_type_2"], axis=1)
# _temp.head()

In [15]:
from sklearn import preprocessing
def encode_categorical(dt, cols):
    for col in cols:
        # Leave NaN as it is.
#         le = preprocessing.LabelEncoder()
#         not_null = df[col][df[col].notnull()]
#         df[col] = pd.Series(le.fit_transform(not_null), index=not_null.index)
        
#         np.save(f'label_class_{col}.npy', le.classes_)
        dt[col] = dt[col].astype("category").cat.codes.astype("int16")
        dt[col] -= dt[col].min()
    return dt


data = encode_categorical(data, ["cat_id", "dept_id", "item_id", "state_id", "store_id"]).pipe(reduce_mem_usage)
values = {'event_name_1': "normal", 'event_type_1': "normal", "event_name_2": "normal", 'event_type_2': "normal"}
data.fillna(value=values, inplace = True);
data = encode_categorical(data, ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]).pipe(reduce_mem_usage)

Mem. usage decreased to 964.68 Mb (4.8% reduction)
Mem. usage decreased to 622.77 Mb (7.3% reduction)


In [16]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-04-26,11513,30,4,1,1,0,0,0,8.257812
1,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,train,2015-04-27,11513,30,4,1,1,0,0,0,8.257812
2,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-04-28,11513,30,4,1,1,0,0,0,8.257812
3,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-04-29,11513,30,4,1,1,0,0,0,8.257812
4,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-04-30,11513,30,4,1,1,0,0,0,8.257812


In [17]:
def datetime_features(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    attrs = [
        "year", 
        "quarter", 
        "month", 
        "week", 
        "day", 
        "dayofweek", 
        "weekday",
        "weekofyear",
#         "is_year_end", 
#         "is_year_start", 
#         "is_quarter_end", 
#         "is_quarter_start", 
#         "is_month_end",
#         "is_month_start",
    ]

    for attr in attrs:
        dtype = np.int16 if attr == "year" else np.int8
        df[attr] = getattr(df['date'].dt, attr).astype(dtype)
    df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(np.int8)
    
    return df

data = datetime_features(data)
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,...,sell_price,year,quarter,month,week,day,dayofweek,weekday,weekofyear,is_weekend
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-04-26,11513,...,8.257812,2015,2,4,17,26,6,6,17,1
1,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,train,2015-04-27,11513,...,8.257812,2015,2,4,18,27,0,0,18,0
2,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-04-28,11513,...,8.257812,2015,2,4,18,28,1,1,18,0
3,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-04-29,11513,...,8.257812,2015,2,4,18,29,2,2,18,0
4,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0,train,2015-04-30,11513,...,8.257812,2015,2,4,18,30,3,3,18,0


In [18]:
_temp = data
_temp.sort_values(by=['id', "date"], inplace=True)
X_train = _temp[_temp["part"]=="train"]
X_val = _temp[_temp["part"]=="validate"]
X_eval = _temp[_temp["part"]=="evaluate"]

print(len(X_train), len(X_val), len(X_eval))

11096813 853720 853720


In [19]:
import math

def numerical_feature(df):
#     for i in [1, 2, 3, 7, 28]:
    for i in [7, 28]:
        df[f"shifted_t{i}"] = df[["id","demand"]].groupby('id')["demand"].shift(i)

#     for i in range(1, 3):
#         df[f"shifted_price_t{i}"] = df.groupby('id')["sell_price"].shift(i)

#     df["price_changes"] = df["shifted_price_t2"] - df["shifted_price_t1"]


#     for win, col in [(7, "demand"), (28, "demand"), (7, "shifted_t7"), (7, "shifted_t28"), (28, "shifted_t7"), (28, "shifted_t28")]:
    for win, col in [(7, "shifted_t7"), (7, "shifted_t28"), (28, "shifted_t7"), (28, "shifted_t28")]:
        df[f"rolling_mean_{col}_w{win}"] = df[["id", col]].groupby('id')[col].shift(1).rolling(win, min_periods=1).mean()
        
#         df[f"rolling_std_i{i}"] = df.groupby('id')["demand"].shift(1).rolling(i, min_periods=1).std()
#         df[f"rolling_skew_i{i}"] = df.groupby('id')["demand"].shift(1).rolling(i, min_periods=1).skew()
#         df[f"rolling_kurt_i{i}"] = df.groupby('id')["demand"].shift(1).rolling(i).kurt()
#         df[f"rolling_max_i{i}"] = df.groupby('id')["demand"].shift(1).rolling(i, min_periods=1).max()
#         df[f"rolling_min_i{i}"] = df.groupby('id')["demand"].shift(1).rolling(i, min_periods=1).min()

#         df[f"price_max_t{i}"] = df.groupby('id')["sell_price"].shift(1).rolling(i, min_periods=1).max()

#     df["rolling_decay_mean"] = df["shifted_t1"].copy()
#     for i in range(2, 8):
#         df["rolling_decay_mean"] += math.pow(0.9, i-1) * df[f"shifted_t{i}"]
#     df["rolling_decay_mean"] = df["rolling_decay_mean"]/7.0

#     weekly_sale = df.groupby(['id', "wm_yr_wk"])["demand"].sum().reset_index()
#     weekly_sale["prev_weekly_sale"] = weekly_sale.groupby('id')["demand"].shift(1)
#     weekly_sale.drop(["demand"], axis=1, inplace=True)
#     df = pd.merge(df, weekly_sale, on=["id", "wm_yr_wk"], how="left")

#     df["acc_sale_by_week"] = df.groupby(['id', "wm_yr_wk"])["demand"].cumsum()
    
    return df



## Avoid Memory crash
I built features on a tiny chunk then aggregate together in my local machine

In [20]:
# It's too big to run
# X = numerical_feature(X_train)
# !mkdir feat

In [21]:
import time

ids = X_train["item_id"].unique()


X = []
start_time = time.time()
for idx, rowId in enumerate(ids):
    if idx%100==1:
        elapsed_time = time.time() - start_time
        start_time = time.time()
        print(idx, elapsed_time)
        
    tinyX = X_train[X_train["item_id"]==rowId]
    _x = numerical_feature(tinyX)
    _x.dropna(inplace = True)
    
    _x.to_csv(f"feat2/numeric_feat{idx}.csv")
    X.append(_x)
    

X = pd.concat(X)

1 0.11799860000610352
101 11.928991794586182
201 11.798969507217407
301 11.66700005531311
401 11.620031356811523
501 11.854963779449463
601 12.071004390716553
701 12.388000726699829
801 12.98499608039856
901 13.529000282287598
1001 13.388004064559937
1101 15.126998901367188
1201 14.12000036239624
1301 12.71204423904419
1401 11.922992467880249
1501 12.050008058547974
1601 12.02895712852478
1701 12.39048147201538
1801 12.533005475997925
1901 12.289960384368896
2001 13.587998867034912
2101 15.715003252029419
2201 14.301997900009155
2301 14.46700119972229
2401 15.397843837738037
2501 15.721996784210205
2601 12.515965938568115
2701 11.656038761138916
2801 11.685961484909058
2901 12.640035152435303
3001 12.310011863708496


In [22]:
# Y = X["demand"]
# X = X.drop(["demand", "part", "date", "wm_yr_wk"], axis=1)
X_val.to_csv(f"feat2/validation.csv")